# Иерархия моделей

1. Посчитать модель линейной регрессии по первым 100 зданиям.
2. Найти точность, используя только дни недели и праздники, примемяя fit_intercept=False и логарифмируя целевой показатель.

Для вычисления отсутствующих или некорректных данных построить модели по всем зданиям одного типа в одном и во всех городах.

## Подключение библиотек

In [1]:
import pandas as pd
import numpy as np

from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

from core.reduce_mem_usage import reduce_mem_usage

## Загрузка данных

In [2]:
buildings = pd.read_csv("../data/buildings.csv")
weather = pd.read_csv("../data/weather.csv")
energy = pd.read_csv("../data/train.0.csv")

## Отбор 100 зданий, объединение и оптимизация

In [3]:
energy = energy[energy["building_id"]<100]
energy = pd.merge(
    left=energy,
    right=buildings,
    how="left",
    left_on="building_id",
    right_on="building_id",
)
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(
    left=energy,
    right=weather,
    how="left",
    left_index=True,
    right_index=True,
)
energy.reset_index(inplace=True)
energy = energy.drop(
    columns=[
        "meter",
        "year_built",
        "square_feet",
        "floor_count",
    ],
    axis=1
)

del buildings
del weather

energy = reduce_mem_usage(energy)
print(energy.info())

Потребление памяти меньше на - 56.89 Мб (минус 71.9%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864557 entries, 0 to 864556
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           864557 non-null  datetime64[ns]
 1   site_id             864557 non-null  int8          
 2   building_id         864557 non-null  int8          
 3   meter_reading       864557 non-null  float16       
 4   primary_use         864557 non-null  category      
 5   air_temperature     864263 non-null  float16       
 6   cloud_coverage      487693 non-null  float16       
 7   dew_temperature     864263 non-null  float16       
 8   precip_depth_1_hr   864459 non-null  float16       
 9   sea_level_pressure  856210 non-null  float16       
 10  wind_direction      839970 non-null  float16       
 11  wind_speed          864557 non-null  float16       
dtypes: category(1), datetime64[ns](1

## Обогащение данных: час, дни недели, праздники

In [4]:
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")

for weekday in range(0, 7):
    energy[f"is_wday {str(weekday)}"] = energy["weekday"].isin([weekday]).astype("int8")

energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start="2015-12-31", end="2017-01-01")
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy["is_holiday"] = energy["date"].isin(us_holidays).astype("int8")